# I) Парсинг

Подгружаем нужные библиотеки

In [2]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns

Собираем ссылки на объявления

In [ ]:
links = []
for i in range(100):
    #будем рабоать со страницей + номер страницы:
    link = 'https://moscow.drom.ru/auto/all/page' + str(i)
    
    #для начала отправляем запрос
    responce = requests.get(link).text

    #создаем объект (в качестве аргументов - значение аштимл и с помощью классов мы можем находить нужное значение на странице)
    soup = bs(responce, 'lxml')

    #достаем наш блок с данными, на странице объявления в нескольких блоках, поэтому будем использовать find_all
    block = soup.find_all('div', class_ = "css-1nvf6xk eojktn00")
    
    #служебный массив
    mass = []
    
    #перебираем в block и ищем наш разделитель - а, в нем получаем href и записываем в служебный массив
    for i in block:
        a = i.find_all('a')
        for j in a:
            mass.append(j.get('href'))
            
    #срезаем лишнее
    mass = mass[:20]
    
    #записываем наши ссылки в конечный массив
    for i in mass:
        links.append(i)

In [ ]:
#создаем нашу таблицу
df = pd.DataFrame(columns = ['Название', 
                             'Год', 
                             'Топливо', 
                             'Объем двигателя', 
                             'Мощность', 
                             'Коробка передач', 
                             'Привод', 
                             'Цвет', 
                             'Пробег',
                             'Руль',
                             'Оценка Drom',
                             'Цена'])

In [ ]:
#собираем данные с объявлений и вносим все в таблицу

#перебираем все ссылки в нашем массиве и создаем наш soup для каждой ссылки
for link in links:
    responce = requests.get(link).text
    soup = bs(responce, 'lxml')
    
    #проверяем не снято ли объявление с продажи, если снято - проходим мимо
    if 'Автомобиль снят с продажи' in str(soup.find('div', class_ = 'css-1bw6vfx edsrp6u2')):
        continue 
    #находим в soup блок с характеристиками и переделываем в строку
    block = str(soup.find('tbody'))

    #определяем название и год
    col = str(soup.find('span', class_ = 'css-1kb7l9z e162wx9x0'))[44:-20].split(', ')
    #случается такое, что по воле божей питон перестает на некоторых объявлениях видеть название. ограничиваем это
    if len(col) == 1:
        continue
    name = col[0]                     #название
    year = int(col[1])                   #год

    #работаем с данными по двигателю
    idx = block.find('Двигатель')
    if idx != -1: 
        engine = block[idx:]
        engine = engine[523:]
        engine = engine[:engine.find('<')]
    else:
        engine = ''
    if 'ГБО' in engine:
        engine = engine[:-5]
    if 'гибрид' in engine:
        engine = engine[:-8]
    mass_fuel = ['бензин', 'дизель', 'электро']
    fuel = ''
    for i in mass_fuel:
        if i in engine:
            fuel = i                                                #здесь определили топливо
            engine = engine.replace(i, '')[2:]
    if len(engine) != 0:
        volume = float(engine[:-2])                                 #здесь объем
    else:
        volume = ''


    #работаем по мощности
    power = block[block.find('Мощность'):]
    power = power[power.find('class="css-9g0qum e162wx9x0">'):][29:]
    power = power[:power.find('<!')]
    if len(power) >= 1:
        if power[0].isdigit() and power[-1].isdigit():
            power = int(power)
    else:
        power = ''

    #работаем по коробке передач
    idx = block.find('Коробка передач')
    transmission = ''
    if idx != -1: 
        transmission = block[idx:][52:]
        transmission = transmission[:transmission.find('<')]

    #определяем привод
    idx = block.find('Привод')
    drive = ''
    if idx != -1: 
        drive = block[idx:][43:]
        drive = drive[:drive.find('<')]

    #определяем цвет
    idx = block.find('Цвет')
    color = ''
    if idx != -1: 
        color = block[idx:][41:]
        color = color[:color.find('<')]

    #определяем руль
    idx = block.find('Руль')
    wheel = ''
    if idx != -1: 
        wheel = block[idx:][41:]
        wheel = wheel[:wheel.find('<')]

    #определяем пробег
    block = soup.find('span', class_ = 'css-1osyw3j ei6iaw00')
    mileage_mass = str(block)[35:-7].split('\xa0')

    mileage = ''
    for i in mileage_mass:
        mileage += i

    for i in mileage:
        if i.isdigit() == False:
            mileage = mileage[:mileage.find('i')]

    #определяем цену
    price_prep = str(soup.find('div', class_ = 'css-eazmxc e162wx9x0'))[34:-24].split('\xa0')
    price = ''
    for i in price_prep:
        price += i
    price = int(price)

    #определяем оценку авто по мнению дром
    score = str(soup.find('div', direction='column', class_ = 'css-obi42x eb0sjjd0'))[239:-6]

    #записываем собранные данные в список
    mass = [name, year, fuel, volume, power, transmission, drive, color, mileage, wheel, score, price]
    #добавляем список к таблице
    df.loc[len(df.index)] = mass

In [ ]:
#сохраняем нашу таблицу
df.to_csv('auto3.csv')